# First Iteration Code Development and Testing

## Algorithmic Trading - Applying Moving Average

### Importing Libraries and Dependencies

In [196]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import warnings
warnings.simplefilter(action="ignore")

In [197]:
# Input Parameters

# Window runs: Short Term (9-21, 21-50, 9-50)
# Long Term: 50-100, 21-100, Golden Cross - Death Cross: 50-200
# Objective: Which of the above combinations gives the best return for each stock

# Set the short window and long windows
short_window = 50
long_window = 200

# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

### Importing Source CSV as Dataframe

In [198]:
# Reading CSV
# Set the file path
filepath = Path("../Resources/SI=F.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
temp_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
btc_df = temp_df.dropna()

# Print the DataFrame
btc_df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-13,15.640,15.655,15.630,15.635,15.635,10541.0
1,2019-01-14,15.590,15.615,15.590,15.609,15.609,38.0
2,2019-01-15,15.610,15.625,15.546,15.546,15.546,64.0
3,2019-01-16,15.535,15.600,15.535,15.564,15.564,7.0
4,2019-01-17,15.467,15.510,15.467,15.467,15.467,27.0
5,2019-01-18,15.330,15.330,15.290,15.330,15.330,14.0
6,2019-01-20,15.375,15.375,15.360,15.365,15.365,17360.0
7,2019-01-22,15.295,15.295,15.140,15.259,15.259,16.0
8,2019-01-23,15.310,15.320,15.280,15.317,15.317,10.0
9,2019-01-24,15.238,15.270,15.238,15.238,15.238,7.0


### Generating Moving Average Crossover Signal

In [199]:
# Grab just the `date` and `close` from the dataset
signals_df = btc_df.loc[:, ["Date", "Close"]].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA09"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA21"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0


signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA09"][short_window:] > signals_df["SMA21"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA09,SMA21,Signal,Entry/Exit
Date,,,,,
2019-01-13,15.635,NaN,NaN,0.0,NaN
2019-01-14,15.609,NaN,NaN,0.0,0.0
2019-01-15,15.546,NaN,NaN,0.0,0.0
2019-01-16,15.564,NaN,NaN,0.0,0.0
2019-01-17,15.467,NaN,NaN,0.0,0.0
2019-01-18,15.330,NaN,NaN,0.0,0.0
2019-01-20,15.365,NaN,NaN,0.0,0.0
2019-01-22,15.259,NaN,NaN,0.0,0.0
2019-01-23,15.317,NaN,NaN,0.0,0.0


 ### Plotting Entry/Exit Points of Moving Average Crossover Trading Strategy

In [200]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA09', 'SMA21']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [201]:
# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

### Backtest the Trading Strategy

In [202]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 500 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df["Close"] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["Close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["Portfolio Cumulative Returns"] = (
    1 + signals_df["Portfolio Daily Returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.tail()

,Close,SMA09,SMA21,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2021-01-06,26.973000,24.72172,22.172260,1.0,0.0,500.0,0.0,13486.5000,88229.0005,101715.5005,-0.002931,0.017155
2021-01-07,27.200001,24.77242,22.231935,1.0,0.0,500.0,0.0,13600.0005,88229.0005,101829.0010,0.001116,0.018290
2021-01-08,24.582001,24.77152,22.277065,1.0,0.0,500.0,0.0,12291.0005,88229.0005,100520.0010,-0.012855,0.005200
2021-01-11,25.239000,24.78878,22.329170,1.0,0.0,500.0,0.0,12619.5000,88229.0005,100848.5005,0.003268,0.008485
2021-01-12,25.384001,24.80594,22.379705,1.0,0.0,500.0,0.0,12692.0005,88229.0005,100921.0010,0.000719,0.009210


### Plot Entry/Exit Points of Backtest Results

In [203]:
# Visualize exit position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [204]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [205]:
### Calculate and Assign Portfolio Evaluation Metrics

In [206]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']]
sortino_ratio_df.loc[:,'Downside Returns'] = 0
sortino_ratio_df.head()

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

,Backtest
Annual Return,0.00450201
Cumulative Returns,0.00921001
Annual Volatility,0.0326216
Sharpe Ratio,0.138007
Sortino Ratio,0.179021


### Prepare Trade Evaluation Metrics DataFrame

In [207]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [208]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss =  exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'Silver',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,Silver,2019-09-10,2020-03-17,500.0,18.041,12.469,9020.5,6234.5,-2786.0


In [209]:
### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [210]:
price_df = signals_df[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

KeyError: "['SMA100', 'SMA50'] not in index"

In [ ]:
### Create Hvplot Table of Portfolio Metrics

In [ ]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

In [ ]:
### Create Hvplot Table of Trade Metrics

In [ ]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table